In [1]:
deployment = "gpt4"
model = "gpt-4"

In [2]:
!pip install chromadb

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


# 加载文档 （Document Loader）

In [2]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('./', glob="spotmax_intro.txt")
docs = loader.load()

In [22]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Spotmax_intro_cn_2020.pdf")
docs = loader.load()

# 切割文档
这个文本分割器是通用文本的推荐分割器。它由一个字符列表参数化。它会按顺序尝试在这些字符上进行分割，直到分块足够小。默认列表是["\n\n", "\n", " ", “"]。这样做的效果是尽可能地保持所有段落（然后是句子，然后是单词）在一起。

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

In [5]:
splits = text_splitter.split_documents(docs)

In [6]:
persist_directory = 'data/'
!rm -rf ./data  

# 将数据存入向量数据库

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embedding = OpenAIEmbeddings(deployment="embedding",chunk_size=1)
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

Using embedded DuckDB with persistence: data will be stored in: data/


In [8]:
docs = vectordb.similarity_search("如何提高可用性",k=2)

In [9]:
docs[0]

Document(page_content='MaxGroup：根据服务⼯作模式和运⾏特征，优化主机群集。智能弹性集群管理可以有效减少弹性计算资源波动（如：竞价实例的中断）带来的集群服务能⼒变化。并在⾯向成本优化构建的混合机型集群最⼤发挥集群不同机型的服务能⼒ 以上各部分将以咨询服务和软件产品的形式提供。', metadata={'source': 'Spotmax_intro_cn_2020.pdf', 'page': 0})

# 利用向量数据库进行QA

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/retrieval_qa/prompt.py

In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=vectordb.as_retriever(),return_source_documents=False)



In [11]:
result = qa({"query": "什么是SpotMax"})
result["result"]


'SpotMax是一个利用弹性计算资源进行云端系统成本优化的体系（包括相关产品及解决方案）。SpotMax旨在帮助用户使用云平台提供的弹性资源（如：AWS Spot Instance）在保证系统可用性，甚至获得更高可用性的前提下，实现成本的最大化节省。SpotMax来自于Mobvista降低成本的成功实践。'